In [1]:
import requests,urllib,math
from urllib.parse import urlparse, parse_qs
from itertools import zip_longest
from datetime import datetime

time = datetime.fromtimestamp

import sqlite3,requests
from bs4 import BeautifulSoup
headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    }
login_data = {
        'do': "scplogin",
        'userid': "sandeep.kumar@idrive.com",
        'passwd': "",
        'submit': ''
        }

queryURL ='https://ticket.idrive.com/scp/tickets.php?sort=date&dir=0&a=search&search-type=&query='

exclude = ['India Billing Support','India Support','Indiasupport Supervisors','Indiasupport SPAM',\
           'IndiaSupport Crisis','IDrive Support','IBackup Support','IndiaSupport Review',\
           'RemotePC Support','Supervisors','US Support','US Development','Sales',\
           'BMR Support','BMR Critcal','Infra Maintenance','BMR DevOps','Finance','BizDev',\
           'Returns','Dev','Spam','Support','Artificial Intelligence','Design','Office IT','GDPR',\
           '— Select —','Express','IndiaSupport 360'
          ]
exclude = set(exclude)
selector1 = ".thread-entry.note .header b"
selector2 = ".thread-entry.note .header time"
selector4 = ".thread-entry.response .header b"
selector5 = ".thread-entry.response .header time"
selector3 = ".thread-event.action .faded.description:has(>b:first-child):has(>time):has(>strong)"
selector7 = ".thread-entry.message .header time"
selector6 = ".thread-entry.message .header b"

depart_head_selector = "div#content > :nth-child(3) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(3) >:nth-child(1)"
depart_name_selector = "div#content > :nth-child(3) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(3) >:nth-child(2)"

assigned_agent_selector = "div#content > :nth-child(5) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(1) >:nth-child(2)"

def ticket_session(name,secret):
    global headers
    with requests.Session() as s:
        url = "https://ticket.idrive.com/scp/login.php"
        try:
            r = s.get(url,headers=headers)
            if r.status_code == 403:
                return 403 # unauthorized
            else:
                return build_header(r,s,url,name,secret)  
        except requests.exceptions.HTTPError as errh:
                return (request_errors[0],errh)
        except requests.exceptions.ConnectionError as errc:
                return (request_errors[1],errc)
        except requests.exceptions.Timeout as errt:
                return (request_errors[2],errt)
        except requests.exceptions.RequestException as err:
                return (request_errors[3],err)
            
def build_header(r,s,url,username,password):
                global login_data
                try:
                    soup = BeautifulSoup(r.content,'html5lib')
                    login_data['__CSRFToken__'] = soup.find('input',attrs={'name':'__CSRFToken__'})['value']
                    login_data['userid']= username
                    login_data['passwd']= password
                    r = s.post(url,data=login_data,headers=headers)
                    if r.status_code == 200:
                        return s
                    else:
                    
                        return 422 # login failed
                except Exception as e:
                    return ('DB Exception-2',e)

In [2]:
def search_query(s,ticketURL=None,searchQuery=None):
    try:
        if searchQuery is not None:
            query = '\"{}\"'.format(urllib.parse.quote_plus(searchQuery.strip()))
            url = queryURL + query
        else:
            url = ticketURL
        r  = s.get(url,headers=headers)
        return r
    except requests.exceptions.HTTPError as errh:
                return 'error-http'
    except requests.exceptions.ConnectionError as errc:
                return 'error-connection'
    except requests.exceptions.Timeout as errt:
                return 'error-timeout'
    except requests.exceptions.RequestException as err:
                print(err)
                return 'error-requestionException'
    except Exception as e:  
        return e

In [3]:
session = ticket_session('sandeep.kumar@idrive.com','sandy86kumar')
html = search_query(session,ticketURL='https://ticket.idrive.com/scp/tickets.php?id=1315669')
soup = BeautifulSoup(html.content,'html5lib')
# print(type(soup))
# soup.select(assigned_agent_selector)[0].get_text().split("Support /")[-1].strip()

In [4]:
def find_responses(soup,selector1,selector2):
    responses = []
    for b,time in zip_longest(soup.select(selector1),soup.select(selector2)):
#         print(b.get_text().strip(),time.get_text().strip())
        responses.append({"agent":b.get_text().strip(),\
                      "time":time.get_text().strip(),\
                      "timestamp": datetime.strptime(time.get_text().strip(),\
                                 '%m/%d/%y, %I:%M %p').timestamp()    
                     })
    return responses

In [5]:
def extract_ticket_url(soup,ticket):
    url = ""
    for tr in soup.select(".list tbody tr"):
        for td in tr.select("td:nth-child(2)"):
            if(td.get_text().strip() == ticket):
                url = "https://ticket.idrive.com/scp/" + td.find_all("a")[0]['href']
                break
    return url

In [6]:

# ticketSearch = search_query(session, searchQuery ='ID631687257')
# searchSoup = BeautifulSoup(ticketSearch.content,'html5lib')
# ticketURL = extract_ticket_url(searchSoup,'ID631687257')
# ticket = search_query(session,ticketURL)
# soup = BeautifulSoup(ticket.content,'html5lib')

In [7]:

def find_notes(soup,selector1,selector2):
    # .thread-entry.message .header  --> user response
    # .thread-entry.note .header --> notes
    # .thread-event.action .faded.description  --> thread action    
    # .thread-entry.response .header     --> agent response
    notes = []
    for b,time in zip_longest(soup.select(selector1),soup.select(selector2)):
#         print(b.get_text().strip(),time.get_text().strip())
        notes.append({"agent":b.get_text().strip(),\
                      "time":time.get_text().strip(),\
                      "timestamp": datetime.strptime(time.get_text().strip(),\
                                 '%m/%d/%y, %I:%M %p').timestamp()    
                     })
    return notes

In [8]:

def find_ticket_events(soup,selector):
    events = []
    for strong in soup.select(selector):
        if("transferred this to" in strong.get_text().strip()):
            item = {  "agent"     :   strong.select("b")[0].get_text().strip(),\
                      "department":   strong.select("strong")[0].get_text().strip(),\
                      "time"      :   strong.select("time")[0].get_text().strip(),\
                      "timestamp" :   datetime.strptime(strong.select("time")[0].get_text().strip(),\
                                     '%m/%d/%y, %I:%M %p').timestamp()  
                    }
            events.append(item)
    return events

In [9]:
def find_responses(soup,selector1,selector2):
    responses = []
    for b,time in zip_longest(soup.select(selector1),soup.select(selector2)):
#         print(b.get_text().strip(),time.get_text().strip())
        responses.append({"agent":b.get_text().strip(),\
                      "time":time.get_text().strip(),\
                      "timestamp": datetime.strptime(time.get_text().strip(),\
                                 '%m/%d/%y, %I:%M %p').timestamp()    
                     })
    return responses

In [10]:
def find_messages(soup,selector1,selector2):
    messages = []
    for b,time in zip_longest(soup.select(selector1),soup.select(selector2)):
#         print(b.get_text().strip(),time.get_text().strip())
        messages.append({"agent":b.get_text().strip(),\
                      "time":time.get_text().strip(),\
                      "timestamp": datetime.strptime(time.get_text().strip(),\
                                 '%m/%d/%y, %I:%M %p').timestamp()    
                     })
    return messages

In [ ]:
notes = find_notes(soup,selector1,selector2)
events = find_ticket_events(soup,selector3)
responses = find_responses(soup,selector4,selector5)
messages = find_messages(soup,selector6,selector7)
find_res_pending(responses,messages)

In [11]:
def find_res_pending(responses,messages):
    i = 0
    res_len = len(responses)
    no_response = None
    if messages:
        msg = messages.pop()
        while i < res_len:
            res = responses.pop()
            if msg['timestamp'] > res['timestamp']:
                no_response = msg
            else:
                break
            i += 1
        return str(datetime.now() - time(no_response['timestamp'])) if no_response else "Updated"
    return "Updated"

In [ ]:
# allEvents = []
# allEvents.extend(notes)
# allEvents.extend(events)
# allEvents.extend(responses)
# allEvents.extend(messages)

In [ ]:
# allEvents.sort(key=lambda x:x['timestamp'])

In [ ]:
# notes = find_notes(soup,selector1,selector2)
# events = find_ticket_events(soup,selector3)

In [12]:
# parse_result = urlparse(ticketURL)
# ticketID = parse_qs(parse_result.query)['id'][0]

# agentsURL = "https://ticket.idrive.com/scp/ajax.php/tickets/" + ticketID + "/assign/agents"
# departmentURL = "https://ticket.idrive.com/scp/ajax.php/tickets/" + ticketID + "/transfer"

def agents_in_department(url):
    ticket = search_query(session,url)
    soup = BeautifulSoup(ticket.content,'html5lib')
    dropdownitems = []
    for select in soup.select(".form-simple select"):
        for option in select.find_all("option"):
            department = option.get_text().strip().split("Support /")[-1].strip()
            if department not in exclude:
                dropdownitems.append(department)
    
    return dropdownitems

In [13]:
def get_agents_or_departments(session,url):
    ticket = search_query(session,url)
    soup = BeautifulSoup(ticket.content,'html5lib')
    dropdownitems = []
    for select in soup.select(".form-simple select"):
        for option in select.find_all("option"):
            department = option.get_text().strip().split("Support /")[-1].strip()
            if department not in exclude:
                dropdownitems.append(department)
    return dropdownitems


In [14]:
def find_last_escalated_item(events,department):
    i = 0
    escalated = None
    events_len = len(events)
    while i < events_len:
        item = events.pop()
        if item["department"] in department:
            escalated = item
            break
        i += 1
    return escalated

In [15]:
import csv
def extract_departments(csvFilePath):
    img_obj = {}
    columns = ['ticket','department']
    try:
        with open(csvFilePath,'r',encoding='utf-8') as file:
            reader = csv.reader(file,delimiter=',')
            for row in reader:
                if row != columns:
                    img_obj[row[1]] = row[0]
        return img_obj
    except Exception as e:
        return e

In [16]:
departments = extract_departments("./departments.csv")
departments

{'IBWin (EVS)': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319742/assign/agents',
 'DavManager (Web)': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319744/assign/agents',
 'IDrive 360': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319746/assign/agents',
 'IDrive E2': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319747/assign/agents',
 'IDriveSync': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319748/assign/agents',
 'IBent (Legacy IDrive/IBackup)': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319749/assign/agents',
 'Linux': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319751/assign/agents',
 'Mobile': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319758/assign/agents',
 'RemotePC Development': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319759/assign/agents',
 'Remote Desktop Support': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319760/assign/agents',
 'RemotePC Codec': 'https://ticket.idrive.com/scp/ajax.php/tickets/1319762/assign/agents',
 

In [ ]:
# def extract_agents(department,visited):
#     ticket = departments[department]
#     ticketSearch = search_query(session,searchQuery=ticket)
#     searchSoup = BeautifulSoup(ticketSearch.content,'html5lib')
#     ticketURL = extract_ticket_url(searchSoup,ticket)
#     ticketContent = search_query(session,ticketURL)
#     parse_result = urlparse(ticketURL)
#     ticketID = parse_qs(parse_result.query)['id'][0]
#     agentsURL = "https://ticket.idrive.com/scp/ajax.php/tickets/" + ticketID + "/assign/agents"
#     agents = set(agents_in_department(agentsURL))
#     visited[department] = agents
#     return agents
    

In [17]:
def extract_agents(session,departments,department,visited):
    try:
        agentsURL = departments[department]
        agents = set(get_agents_or_departments(session,agentsURL))
        visited[department] = agents
        return agents
    except Exception as e:
        return None

In [18]:
def get_ticket_url(department):    
    ticket = departments[department]
    ticketSearch = search_query(session,searchQuery=ticket)
    searchSoup = BeautifulSoup(ticketSearch.content,'html5lib')
    ticketURL = extract_ticket_url(searchSoup,ticket)
    parse_result = urlparse(ticketURL)
    ticketID = parse_qs(parse_result.query)['id'][0]
    agentsURL = "https://ticket.idrive.com/scp/ajax.php/tickets/" + ticketID + "/assign/agents"
    return agentsURL

In [ ]:
# def find_due_date(escalated,notes,ticket,visited):
#     i = 0
#     dev = None
#     diff = None
#     updateDuration = None
#     notes_len = len(notes)
#     if escalated is not None:
#         if escalated['department'] not in visited:
#              extract_agents(escalated['department'],visited)
#     while i < notes_len:
#         item = notes.pop()
#         if escalated is not None and item['timestamp'] > escalated['timestamp'] and \
#             item['agent'] in visited[escalated['department']]:
                
#             updateDuration = datetime.fromtimestamp(item['timestamp']) - \
#             datetime.fromtimestamp(escalated['timestamp'])
#             dev = item
#             break
#         i += 1
#     if updateDuration is None and escalated is not None:
#         diff = datetime.now() - datetime.fromtimestamp(escalated['timestamp'])
        
#     return {"ticket":ticket, \
#             "due":str(diff),\
#             "escalated":escalated,\
#             "ifUpdatedTimeTaken":str(updateDuration),\
#             "updatedAgent":dev
#            }

In [31]:
def find_due_date(session,departments,escalated,\
                notes,ticket,visited,assigned_agent,responses):
    i = 0
    dev = None
    diff = None
    updateDuration = None
    agent_res = None
    agent_res_time = None
    dev_update_time = None
    agent_update_pending = None
    notes_len = len(notes)
    responses_len = len(responses)
    j = 0
    
    if "India Support" not in visited:
        extract_agents(session,departments,"India Support",visited)
    
    
    if escalated is not None:
        if escalated['department'] not in visited:
             extract_agents(session,departments,escalated['department'],visited)   
    poppedNotes = []
    while i < notes_len:
        note = notes.pop()

        if escalated is not None and note['timestamp'] > escalated['timestamp'] and \
            note['agent'] in visited[escalated['department']]:
            updateDuration = datetime.now() - time(note['timestamp'])
            dev_update_time = time(note['timestamp']) 
            j = 0
            while j < responses_len:
                res = responses.pop()
                if res['timestamp'] > note['timestamp']:
                    if (res['agent'] == assigned_agent or res['agent'] in visited['India Support'] ):
                        agent_res = res
                else:
                    break
                j += 1
            
            if agent_res is not None:
                agent_res_time = time(agent_res['timestamp']) - time(note['timestamp'])
            dev = note
            break
        i += 1

    if updateDuration is None and escalated is not None:
        diff = datetime.now() - time(escalated['timestamp'])
    if updateDuration is not None and agent_res is None:
        agent_update_pending = datetime.now() - dev_update_time

    return {"ticket":ticket, \
            "due":str(diff),\
            "escalated":escalated,\
            "Developer Update":str(updateDuration),\
            "updatedAgent":dev,\
            "assignedAgentUpdate" : str(agent_res_time) if agent_res_time else None,\
            "agentUpdatePending" : str(agent_update_pending) if agent_update_pending else None
           }

In [21]:
# balamurugan
# sandeep
# vinil
# ravi

# popped --> ravi,vinil,sandeep -->

In [22]:
# escalated = find_last_escalated_item(events)
# find_due_date(escalated,notes)

In [23]:
# from tqdm import tqdm
departmentURL = "https://ticket.idrive.com/scp/ajax.php/tickets/1319774/transfer"
department = set(agents_in_department(departmentURL))
department

{'DavManager (Web)',
 'IBWin (EVS)',
 'IBent (Legacy IDrive/IBackup)',
 'IDrive 360',
 'IDrive C2C',
 'IDrive E2',
 'IDriveSync',
 'Infra',
 'Linux',
 'Mac',
 'Meeting',
 'Mirror',
 'Mobile',
 'Mobile / Artificial Intelligence',
 'Openstack-Infra',
 'Remote Desktop Support',
 'RemotePC Codec',
 'RemotePC Development'}

In [24]:
def extract_tickets(csvFilePath):
    img_obj = []
    columns = ['ticket']
    try:
        with open(csvFilePath,'r',encoding='utf-8') as file:
            reader = csv.reader(file,delimiter=',')
            for row in reader:
                if row != columns:
                    img_obj.append(row[0])
        return img_obj
    except Exception as e:
        return e

In [25]:
tickets = extract_tickets('./tickets.csv')
len(tickets)

50

In [26]:
visited = {}
info = []
totalDue = []
n_workers = 5
from copy import deepcopy
def process_ticket(ticket):
    ticketSearch = search_query(session, searchQuery = ticket)
    searchSoup = BeautifulSoup(ticketSearch.content,'html5lib')
    ticketURL = extract_ticket_url(searchSoup,ticket)
    ticketContent = search_query(session,ticketURL)
    soup = BeautifulSoup(ticketContent.content,'html5lib')
    notes = find_notes(soup,selector1,selector2)
    responses = find_responses(soup,selector4,selector5)
    deepcopy_responses = deepcopy(responses)
    notes_len = len(notes)
    events = find_ticket_events(soup,selector3)
    events_len = len(events)
    escalated = find_last_escalated_item(events,department)
    assigned_department = soup.select(depart_name_selector)[0].get_text().split("Support /")[-1].strip()
    assigned_agent = soup.select(assigned_agent_selector)[0].get_text().split("Support /")[-1].strip()
    due = find_due_date(session,departments,escalated,notes,ticket,visited,assigned_agent,responses)
    messages = find_messages(soup,selector6,selector7)
    due['UsersResponsePending'] = find_res_pending(deepcopy_responses,messages)
    due['notes'] = notes_len
    due['events'] = events_len
    html = search_query(session,ticketURL=ticketURL)
    soup = BeautifulSoup(html.content,'html5lib')
    due['department'] = assigned_department
    due['assigned'] = assigned_agent
    
    return due
    

In [32]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool
from tqdm import tqdm
info = []
with ThreadPoolExecutor(n_workers) as exe:
    with tqdm(total=len(['ID299724708'])) as pbar:
        futures = [exe.submit(process_ticket,ticket) for ticket in ['ID299724708']]
        for future in futures:
            due = future.result()
            info.append(due)
            pbar.update(1)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.59s/it]


In [33]:
info

[{'ticket': 'ID299724708',
  'due': 'None',
  'escalated': {'agent': 'Oskar Support',
   'department': 'DavManager (Web)',
   'time': '6/9/23, 4:46 AM',
   'timestamp': 1686266160.0},
  'Developer Update': '4 days, 17:44:27.541154',
  'updatedAgent': {'agent': 'Shashi Arroju',
   'time': '6/9/23, 11:00 PM',
   'timestamp': 1686331800.0},
  'assignedAgentUpdate': '2:00:00',
  'agentUpdatePending': None,
  'UsersResponsePending': 'Updated',
  'notes': 3,
  'events': 4,
  'department': 'US Support',
  'assigned': 'Oskar Support'}]

In [ ]:
# visited = {}
# info = []
# totalDue = []
# for ticket in tqdm(tickets):
#     ticketSearch = search_query(session, searchQuery = ticket)
#     searchSoup = BeautifulSoup(ticketSearch.content,'html5lib')
#     ticketURL = extract_ticket_url(searchSoup,ticket)
#     ticketContent = search_query(session,ticketURL)
#     soup = BeautifulSoup(ticketContent.content,'html5lib')
#     notes = find_notes(soup,selector1,selector2)
#     notes_len = len(notes)
#     events = find_ticket_events(soup,selector3)
#     events_len = len(events)
#     escalated = find_last_escalated_item(events,department)
#     due = find_due_date(escalated,notes,ticket,visited)
#     due['notes'] = notes_len
#     due['events'] = events_len
#     info.append(due)
#     totalDue.append([due['ticket'],due["due"]])

In [ ]:
newData = create_dataframe_csv(info)

In [ ]:
# newData = []
# columns = ["Ticket","Due Since","Department","Assigned Agent","Developer","LastUpdated","HasAgentUpdated","Notes","Events"]
# for row in info:
#     newData.append([row['ticket'],\
#                     row["due"],\
#                     row['department'],\
#                     None if row["updatedAgent"] is None else row["updatedAgent"]["agent"],\
#                     None if row["updatedAgent"] is None else \
#                     datetime.now() - datetime.fromtimestamp(row["updatedAgent"]['timestamp']),\
#                     row['notes'],\
#                     row['events']
#                    ])

In [ ]:
columns = ["Ticket","Response to User","Ticket Due","Department","Assigned","Developer","Developer Update","Response to Dev","Response Pending to Dev Update","Notes","Events"]
def create_dataframe_csv(processed_tickets):
    df = []
    for row in processed_tickets:
        df.append([row['ticket'],\
                        row["UsersResponsePending"],\
                        row["due"],\
                        row['department'],\
                        row['assigned'],\
                        None if row["updatedAgent"] is None else row["updatedAgent"]["agent"],\
                        None if row["updatedAgent"] is None else \
                        datetime.now() - datetime.fromtimestamp(row["updatedAgent"]['timestamp']),\
                        row['assignedAgentUpdate'],\
                        row["agentUpdatePending"],\
                        row['notes'],\
                        row['events']
                    ])
        
    return df

In [ ]:
def write_csv(csvFilePath,columns,arr):
    try:
        with open(csvFilePath,'w',newline='',encoding='utf-8') as file:
            writer = csv.writer(file,delimiter=',')
            writer.writerow(columns)
            for row in arr:
                writer.writerow(row)
    except Exception as e:
        return e

In [ ]:
# extract_departments("./departments.csv")

In [ ]:
write_csv('./escalated.csv',columns,newData)

In [ ]:
info[1]

In [ ]:
newData[26]

In [ ]:
info